In [1]:
import os
from PIL import Image
import shutil
import subprocess

# CellProfiler Python Imports
import javabridge
import cellprofiler_core.image
import cellprofiler_core.object
import cellprofiler_core.pipeline
import cellprofiler_core.preferences
import cellprofiler_core.workspace

In [2]:
def convert_file_format(input_folder, output_folder, format = "jpeg"):
    '''
    Parameters:
    1. input_folder - path to tiled .tiff images
    2. output_folder - output directory of newly-created .format images
    3. format - image file format to convert to
    
    Returns: .format (default = jpg) images stored in the same parent directory as the input folder
             but in a different sub-directory (& without overwriting). Naming conventions are 
             updated accordingly (for batch-processing purposes in Ilastik)
    '''
    
    for file in os.listdir(input_folder):
        origFile = file;
        
        # In case tiles were scraped using QuPath script
        file = file.replace('[', '');
        file = file.replace(']', '');
        os.rename(os.path.join(input_folder, origFile), os.path.join(input_folder, file))
    
    for file in os.listdir(input_folder):
        im = Image.open(input_folder + "/" + file);
        im.save(output_folder + "/" + file[:-4] + "." + format, format = format)
        im.close()

In [3]:
def collect_probs(ilastik_project, TB_images):
    '''
    Parameters:
    1. ilastik_project - path to annotated ilastik project
    2. TB_images - folder containing TB-stained images (.jpg) to be analyzed
    
    Returns: Probability maps stored in subdirectory of TB images (to feed into CellProfiler)
    '''
    
    # Runs ilastik project on specified directory of validation images
    input_images = [file for file in os.listdir(TB_images) if "TB" in file]
    
    if len(input_images) != 0:
        os.mkdir(os.path.join(TB_images, "Probability Maps"))
        for file in input_images:
            print(file + " ->", end = " ")
            # Data specs as suggested by CellProfiler
            command = ["/home/rohit/Ilastik/run_ilastik.sh", "--headless", 
                   "--project=" + ilastik_project, "--output_format=tif sequence", 
                   "--output_filename_format=" + TB_images + "/Probability Maps/{nickname}_{result_type}_{slice_index}.tif", 
                   "--export_source=Probabilities", "--export_dtype=uint16", "--output_axis_order=cyx", 
                   "--pipeline_result_drange=(0,65535)", os.path.join(TB_images, file)]
            subprocess.run(command)
            print("Completed!")

        # Operations to organize resulting output directory
        os.mkdir(os.path.join(TB_images + "/Probability Maps", "Slice Index 0"))
        os.mkdir(os.path.join(TB_images + "/Probability Maps", "Slice Index 1"))
        for file in os.listdir(os.path.join(TB_images, "Probability Maps")):
            if (file.endswith("1.tif")):
                shutil.move(TB_images + "/Probability Maps/" + file, 
                            TB_images + "/Probability Maps/Slice Index 1/" + file)
            elif (file.endswith("0.tif")):
                shutil.move(TB_images + "/Probability Maps/" + file, 
                            TB_images + "/Probability Maps/Slice Index 0/" + file)        

In [4]:
def clean_prob_maps(prob_images):
    # Ideas:
    # 1. If average background > 128 (initial guess, might need to tune this parameter), convert tile
    #    to all black (reasoning: highly improbable balloon cells are more prevalent than background)
    # 2. Add certainty percentage that it is a balloon cell in the napari graphic
    # 3. Neighborhood (Chebyshev distance) analysis of pixels to aid in classification
    
    return

In [5]:
# The following code, stripped directly from CellProfiler documentation, does not run properly. After testing 
# thoroughly and looking through past GitHub issues, I am not able to find a current solution. Thus, alternative
# approaches need to be examined (i.e. command line, Python modules, numPy. Ilastik object detection). One such
# solution is presented down below (running from command line with pre-established pipeline).

def analyze_intensities(cellprofiler_pipeline, image_slice):
    '''
    Code adopted from CellProfiler GitHub documentation, CellProfiler/notebooks/cellprofiler_demo.ipynb 
    & "CellProfiler as a Python package"
    
    Parameters:
    1. image_slice - probability maps obtained from Ilastik (Slice Index 0)
    2. cellprofiler_pipeline - path to CellProfiler pipeline (.cppipe file not .cpproj)
    
    Returns: Annotated images with marked balloon cells (red outline)
    '''
    
    cellprofiler_core.preferences.set_headless()
    
    # Create and load the pipeline
    pipeline = cellprofiler_core.pipeline.Pipeline()
    pipeline.load(cellprofiler_pipeline)
    
    # Create the image set and add the image data
    image_set_list = cellprofiler_core.image.ImageSetList()
    image_set = image_set_list.get_image_set(0)
    for image_name in os.listdir(image_slice):
        image = Image.open(os.path.join(image_slice, image_name))
        image_set.add(image_name, cellprofiler_core.image.Image(image))

    # Create an ObjectSet instance and name and add an Objects instance
    object_set = cellprofiler_core.object.ObjectSet()
    objects = cellprofiler_core.object.Objects()
    object_set.add_objects(objects, "BC")

    # Create a Measurements instance
    measurements = cellprofiler_core.measurement.Measurements()
    
    # Create a Workspace instance
    for module in pipeline.modules():
        workspace = cellprofiler_core.workspace.Workspace(
            image_set = image_set,
            image_set_list = image_set_list,
            measurements = measurements, 
            module = module,
            object_set = object_set, 
            pipeline = pipeline
        )
        
        module.prepare_run(workspace)
        module.run(workspace)
        module.post_run(workspace)
        
    return workspace

In [13]:
def cellprofiler_command_line(image_slice, cellprofiler_pipeline, output_folder):
    '''
    Parameters:
    1. image_slice - probability maps obtained from Ilastik (Slice Index 0) & regular TB image
    2. cellprofiler_pipeline - path to CellProfiler pipeline (.cppipe file not .cpproj)
    3. output_folder - where to place annotated CellProfiler images
    
    Returns: Annotated images with marked balloon cells (red outline)
    '''
        
    print(output_folder + " ->", end = " ")
    command = ["cellprofiler", "-c", "-r", "-p", cellprofiler_pipeline, 
               "--file-list", str(image_slice)]
    subprocess.run(command)
    print("Completed!")
    
    # Organize output directory
    os.mkdir(output_folder + "/Annotated")
    for image in os.listdir(output_folder):
        if image.endswith(".tiff"):
            shutil.move(output_folder + "/" + image, output_folder + "/Annotated/" + image)

In [9]:
# Run ilastik on tiled images (make sure to get rid of mask_errors.txt before running!)
ilastik_project = "/home/rohit/Documents/Ilastik Balloon Cells (FINAL)/(new) Balloon Cell Identification.ilp"
tile_location = "/home/rohit/Documents/Ilastik Balloon Cells (FINAL)/Validation Images"

# Use this if images are separated by folder
for file in sorted(os.listdir(tile_location)):
    collect_probs(ilastik_project,
                  os.path.join(tile_location, file))

'''
# Use this if images are all in the same folder
collect_probs(ilastik_project, tile_location)
'''

Validation Image #1 TB.tif -> Completed!
Validation Image #10 TB.tif -> Completed!
Validation Image #11 TB.tif -> Completed!
Validation Image #12 TB.tif -> Completed!
Validation Image #13 TB.tif -> Completed!
Validation Image #14 TB.tif -> Completed!
Validation Image #15 TB.tif -> Completed!
Validation Image #16 TB.tif -> Completed!
Validation Image #17 TB.tif -> Completed!
Validation Image #18 TB.tif -> Completed!
Validation Image #19 TB.tif -> Completed!
Validation Image #2 TB.tif -> Completed!
Validation Image #20 TB.tif -> Completed!
Validation Image #3 TB.tif -> Completed!
Validation Image #4 TB.tif -> Completed!
Validation Image #5 TB.tif -> Completed!
Validation Image #6 TB.tif -> Completed!
Validation Image #7 TB.tif -> Completed!
Validation Image #8 TB.tif -> Completed!
Validation Image #9 TB.tif -> Completed!


'\n# Use this if images are all in the same folder\ncollect_probs(ilastik_project, tile_location)\n'

In [14]:
# Run cellprofiler on ilastik probability maps 
cellprofiler_pipeline = "/home/rohit/Documents/Ilastik Balloon Cells (FINAL)/(new) Compact Analysis.cppipe"
tile_location = "/home/rohit/Documents/Ilastik Balloon Cells (FINAL)/Validation Images"

for folder in os.listdir(tile_location):
    cellprofiler_images = []
    output_folder_root = ""
    
    for root, dirs, files in os.walk(os.path.join(tile_location, folder)):
        if ("Slice Index 0" in root):
            output_folder_root = root
            
        for file in files: 
            if ("TB" in file and "Probabilities_1" not in file):
                cellprofiler_images.append(os.path.join(root, file))
    
    if (len(cellprofiler_images) == 2):
        cellprofiler_images = sorted(cellprofiler_images, reverse = True)
        
        txt_path = os.path.join(os.path.join(tile_location, folder), 
                              "cellprofiler_analysis.txt")
        f = open(txt_path, "w")
        f.write(cellprofiler_images[0])
        f.write("\n")
        f.write(cellprofiler_images[1])
        f.close()
        
        cellprofiler_command_line(txt_path, 
                                  cellprofiler_pipeline, output_folder_root)

/home/rohit/Documents/Ilastik Balloon Cells (FINAL)/Validation Images/Validation Image #18/Probability Maps/Slice Index 0 -> Completed!
/home/rohit/Documents/Ilastik Balloon Cells (FINAL)/Validation Images/Validation Image #9/Probability Maps/Slice Index 0 -> Completed!
/home/rohit/Documents/Ilastik Balloon Cells (FINAL)/Validation Images/Validation Image #19/Probability Maps/Slice Index 0 -> Completed!
/home/rohit/Documents/Ilastik Balloon Cells (FINAL)/Validation Images/Validation Image #6/Probability Maps/Slice Index 0 -> Completed!
/home/rohit/Documents/Ilastik Balloon Cells (FINAL)/Validation Images/Validation Image #16/Probability Maps/Slice Index 0 -> Completed!
/home/rohit/Documents/Ilastik Balloon Cells (FINAL)/Validation Images/Validation Image #20/Probability Maps/Slice Index 0 -> Completed!
/home/rohit/Documents/Ilastik Balloon Cells (FINAL)/Validation Images/Validation Image #15/Probability Maps/Slice Index 0 -> Completed!
/home/rohit/Documents/Ilastik Balloon Cells (FINAL